https://www.quora.com/How-can-a-standard-deviation-divided-by-mean-be-useful 

The coefficient of variation (CV), defined as Standard deviation (SD) divided by the Mean describes the variability of a sample relative to its mean. Because the CV is unitless and usually expressed as a percentage, it is used instead of the SD to compare the spread of data sets that have different units of measurements or have the same units of measurements but differs greatly in magnitude.

Let’s say you’re comparing the weights of mice and rabbits. You determine from a sample that the average weight of mice is 1 ounce with a SD of 0.08 ounces, whereas the mean weight of rabbits is 16 ounces with a SD of 0.4 ounces.

Although the SD of the rabbits is five times greater than the SD of the mice, their CVs support a different conclusion:

•	Rabbits: CV = 100 * 0.4 ounces / 16 ounces = 2.5 %

•	Mice: CV = 100 * 0.08 ounces / 1 ounce = 8 %

•	The CV of mice is more than three times greater than that of the rabbits. In other words, although rabbits have a greater SD, mice have much more weight variability relative to their mean.

For variables measured in different units (let’s say you want to compare weight measured in ounces and length measured in inches) it provides a simple way to compare oranges to apples.

In [1]:
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/')
import netCDF4
import fill_values as fv
import datetime
import calc_date as cd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import save_fig as sF
import createFolder as cF
from matplotlib.gridspec import GridSpec
import plt_ce_image as im
import read_MRR as pMRR
import plot_sfc_spaghetti_ret as spagh

from scipy.integrate import simps

#plt.style.use('ggplot')
##plt.style.use('classic')

In [2]:
year = '2016'
month = '12'
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
tid = '00'
station = 'Haukeliseter'


### Figures 
savefig = 1
figdir = '../../Figures/Retrieval_MEPS/coefficent_variation/_48'
cF.createFolder('%s/' %(figdir))
form = 'png'
################################################################
### MEPS
level = 'sfc'
nc_dir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = 'ml'
nc_dir_ml = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
# air temperature
var_name_air_temp = 'air_temperature_ml'
air_temp_dir = '%s/%s' %(nc_dir_ml,var_name_air_temp)
# snow fall amount
var_name_snow = 'snowfall_amount_ml'
snow_dir = '%s/%s' %(nc_dir_ml,var_name_snow)
# graupel fall amount
var_name_graupel = 'graupelfall_amount_ml'
graupel_dir = '%s/%s' %(nc_dir_ml,var_name_graupel)
# atmosphere_cloud_ice_content_ml
var_name_cl_ice = 'atmosphere_cloud_ice_content_ml'
cl_ice_dir = '%s/%s' %(nc_dir_ml,var_name_cl_ice)

################################################################
### MRR
MRR_dir = '../../Data/MRR/processed_MRR'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################

In [3]:
## MRR ######
fnMRR      = dict()
time_MRR   = dict()
height_MRR = dict()
Ze         = dict()

In [4]:
## RETRIEVAL ######
fnRet      = dict()
retrieved_snowfall_amount = dict()
SWC        = dict()
SWP_ret    = dict() 
SWC_con    = dict()
time_con   = dict()
SWP_mean_ret= dict()

In [5]:
air_temp_file     = dict()
snow_file         = dict()
graupel_file      = dict()
cl_ice_file       = dict()

fn_air_temp       = dict()
fn_snow           = dict()
fn_graupel        = dict()
fn_cl_ice         = dict()

time_ml           = dict()
air_temp_ml       = dict()
pressure_ml       = dict()
snow_amount_ml    = dict()
graupel_amount_ml = dict()
cl_ice_ml         = dict()

thickness_ml      = dict()
ice_amount_ml     = dict()
time              = dict()
height_ml         = dict()
height            = dict()
density_ml        = dict()
SWP_model         = dict()



In [6]:
Rd = 287.    # gas constant for dry air [J kg^-1 K^-1]

In [7]:
for day in t:
## MRR ######  
    fnMRR[day] = netCDF4.Dataset('%s/VMRR_%s%s%s.nc' %(MRR_dir,year,month,day) ,'r')
    time_MRR[day]   = fnMRR[day].variables['time'][:]
    height_MRR[day] = fnMRR[day].variables['height'][:]

    Ze[day] = pMRR.read_and_mask(fnMRR[day],'Ze', np.nan)         # vertical Ze profile for retrieval

## RETRIEVAL ######
    fnRet[day] = netCDF4.Dataset('%s/out_swc_b6_%s%s%s.nc' %(nc_dir_retrieval, year, month, day))

    retrieved_snowfall_amount[day] = fnRet[day].variables['retrieved_snowfall_amount'][:]
    SWC[day] = fnRet[day].variables['SWC'][:]
    
    swpret = []
    swpret = simps(SWC[day][:,:], height_MRR[day], axis = 0)
    SWP_ret[day] = swpret


### hourly SWC 
mean_SWC = dict()
for day in t:
    if day == '20':
        time_con[day]= np.concatenate((time_MRR[day], time_MRR[str(int(day)+1)][:30]))
        SWC_con[day] = np.concatenate((SWC[day], SWC[str(int(day)+1)][:,:30]),axis=1)
    elif day == '27':
        time_con[day]= np.concatenate((time_MRR[str(int(day)-1)][23*60+30:], time_MRR[day]))
        SWC_con[day] = np.concatenate((SWC[str(int(day)-1)][:,23*60+30:], SWC[day]),axis=1)
    else:
        time_con[day]= np.concatenate((time_MRR[str(int(day)-1)][23*60+30:], time_MRR[day], time_MRR[str(int(day)+1)][:30]))
        SWC_con[day] = np.concatenate((SWC[str(int(day)-1)][:,23*60+30:], SWC[day], SWC[str(int(day)+1)][:,:30]),axis=1)
    
    avg60 = []
    if day == '20':
        for i in range(30,SWC_con[day].shape[1],60):
            avg60.append(np.nanmean(SWC_con[day][:,i:(i+60)],axis=1))
        mean_SWC[day] = np.transpose(avg60)
    else:
        for i in range(0,SWC_con[day].shape[1],60):
            avg60.append(np.nanmean(SWC_con[day][:,i:(i+60)],axis=1))
        mean_SWC[day] = np.transpose(avg60)

### hourly SWP
    swpret = []
    swpret = simps(mean_SWC[day][:,:], height_MRR[day], axis = 0)
    SWP_mean_ret[day] = swpret

# for plotting, only values larger than 0.01
    a,b = np.where(retrieved_snowfall_amount[day][:,:] < 0.01)
    SWC[day][a[:],b[:]]                       = np.nan
    retrieved_snowfall_amount[day][a[:],b[:]] = np.nan
    np.transpose(Ze[day])[a[:],b[:]]          = np.nan
#    mean_SWC[day][a[:],b[:]]                  = np.nan
    a1,b1 = np.where(mean_SWC[day][:,:] < 0.01)    
    mean_SWC[day][a1[:],b1[:]] = np.nan
    
    fnMRR[day].close()
    fnRet[day].close()

/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/read_MRR.py:95: RuntimeWarning: invalid value encountered in less
  mask = np.ma.getmaskarray(var[:,:])
/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval/read_MRR.py:96: RuntimeWarning: invalid value encountered in less
  var = np.ma.array(var[:,:], mask = mask, fill_value = fill_value)


In [8]:
# layer for average snow water content within 200 m
nlay= 15
bot = []
top = []
for k in range(0, nlay):
    bot.append(100.+ k*200)
    top.append(bot[k]+200)

avg_SWC_ml = dict()
h_avg      = dict()
t_avg      = dict()

In [9]:
#Xmax = 24
Xmax = 48

In [10]:
for k in range(0,np.asarray(t).shape[0]):
#for k in range(3,4):
    day = t[k]
## MEPS ######
    for ens_memb in range(0,10):
        air_temp_file[ens_memb]= '%s/%s%s%s_%s_%s.nc' %(air_temp_dir,year,month,day,tid,ens_memb)
        snow_file[ens_memb]    = '%s/%s%s%s_%s_%s.nc' %(snow_dir,year,month,day,tid,ens_memb)
        graupel_file[ens_memb] = '%s/%s%s%s_%s_%s.nc' %(graupel_dir,year,month,day,tid,ens_memb)
        cl_ice_file[ens_memb]  = '%s/%s%s%s_%s_%s.nc' %(cl_ice_dir,year,month,day,tid,ens_memb)
        
        fn_air_temp[ens_memb]  = netCDF4.Dataset(air_temp_file[ens_memb])
        fn_snow[ens_memb]      = netCDF4.Dataset(snow_file[ens_memb])
        fn_graupel[ens_memb]   = netCDF4.Dataset(graupel_file[ens_memb]) 
        fn_cl_ice[ens_memb]    = netCDF4.Dataset(cl_ice_file[ens_memb])

# Read in time to know initialisation time (plotting)    
        time_ml[ens_memb] = fn_snow[ens_memb].variables['time']
        time_ml[ens_memb] = fv.fill_nan(time_ml[ens_memb][:])
        

        ini_day = datetime.datetime.utcfromtimestamp(time_ml[0][0]).day   # day of initialisation
        hh = datetime.datetime.utcfromtimestamp(time_ml[0][0]).hour       # first hour of initialisation?
        calday, calmon = cd.get_dayname(year, month, ini_day)
        
# Read in the pressure, and the air temperature to calculate ice amount from [kg/kg] to [kg/m^3]
        air_temp_ml[ens_memb] = fn_air_temp[ens_memb].variables[var_name_air_temp]
        pressure_ml[ens_memb] = fn_snow[ens_memb].variables['pressure_ml']
# Read in the variable name which should be plotted  
        snow_amount_ml[ens_memb]    = fn_snow[ens_memb].variables[var_name_snow]
        graupel_amount_ml[ens_memb] = fn_graupel[ens_memb].variables[var_name_graupel]
        cl_ice_ml[ens_memb]         = fn_cl_ice[ens_memb].variables[var_name_cl_ice]


# create an array with time of the shape of the variable
        lead_time   = np.arange(0,snow_amount_ml[ens_memb].shape[0])
        model_level = np.arange(0,snow_amount_ml[ens_memb].shape[1])
        td,Y        = np.meshgrid(lead_time,model_level)

# substitute missing values with nan 
        pressure_ml[ens_memb]       = fv.fill_nan(pressure_ml[ens_memb][:])
        air_temp_ml[ens_memb]       = fv.fill_nan(air_temp_ml[ens_memb][:])
        snow_amount_ml[ens_memb]    = fv.fill_nan(snow_amount_ml[ens_memb][:])
        graupel_amount_ml[ens_memb] = fv.fill_nan(graupel_amount_ml[ens_memb][:])
        cl_ice_ml[ens_memb]         = fv.fill_nan(cl_ice_ml[ens_memb][:])
# get ice amount (snow+graupel)
        ice_amount_ml[ens_memb]     = snow_amount_ml[ens_memb] + graupel_amount_ml[ens_memb] + cl_ice_ml[ens_memb]

    
# layer thickness to calculate height
        thickness_ml[ens_memb]      = fn_snow[ens_memb].variables['layer_thickness']
        thickness_ml[ens_memb]      = fv.fill_nan(thickness_ml[ens_memb][:])
    
        thickness = thickness_ml[ens_memb]
        h_above = []
        h_above.append(np.zeros(thickness[:,:].shape[0]))
        th_arr = np.nansum([np.asarray(h_above)[:,0], thickness[:,0]], axis = 0)
        h_above.append(th_arr)
        for i in range(2,thickness[:,:].shape[1]):
            th_arr = np.nansum([h_above[i-1], thickness[:,i-1]], axis = 0)
            h_above.append(th_arr)
        height_ml[ens_memb] = np.transpose(h_above) 
        
        
####
# calculate density at each level
        density_ml[ens_memb] = pressure_ml[ens_memb]/(Rd*air_temp_ml[ens_memb])
              

# convert from [kg/kg] to [kg/m^3]
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*density_ml[ens_memb])

# convert the snow amount from [kg/m^2] into [g/m^2] by multiply with 1000
        ice_amount_ml[ens_memb] = (ice_amount_ml[ens_memb]*1000.)
    
    
# calculate mean of snowfall amount in each layer, every 200m
        SWC_ml = []
        h_mid = []
        for k in range(0, nlay):
            tidx,hidx = np.where(np.logical_and(height_ml[ens_memb][:,:] >= bot[k], height_ml[ens_memb][:,:] < top[k]))
# average in layer
            SWC_ml.append(np.nanmean(ice_amount_ml[ens_memb][:,hidx[:]], axis = 1))
            h_mid.append((bot[k]+top[k])/2)
        avg_SWC_ml[ens_memb] = SWC_ml      
        
        time[ens_memb]          = td
        height[ens_memb]        = height_ml[ens_memb]   
        
        hidx = []
        tidx = []
        for i in range(0,np.asarray(avg_SWC_ml[ens_memb]).shape[1]):
            hidx.append(h_mid)
        h_avg[ens_memb] = hidx
        for i in range(0,np.asarray(avg_SWC_ml[ens_memb]).shape[0]):
            tidx.append(time[ens_memb][0,:])
        t_avg[ens_memb] = tidx
        
        
## only use values below 3km
        h1, h2 = np.where(height[ens_memb][:,:] > 3000)
        ice_amount_ml[ens_memb][h1[:],h2[:]] = np.nan
        time[ens_memb][h2[:],h1[:]]          = -9999
        height[ens_memb][h1[:],h2[:]]        = np.nan
        
        
               

    print('%s, %s %s %s' %(calday, day, calmon, year))
    # retrieval 
    r1, r2 = np.where(mean_SWC[day] == np.nanmax(mean_SWC[day] ))
    if len(r1) == 0 or len(r2) == 0:
        print('max SWC in retrieval: NaN')
    else: 
        print('max SWC in retrieval: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(mean_SWC[day]),height_MRR[day][r1],r2))
    

    # MEPS
    ml1, ml2 = np.where(np.transpose(avg_SWC_ml[0]) == np.nanmax(np.transpose(avg_SWC_ml[0])[:25,:] ))
    if len(ml1) == 0 or len(ml2) == 0:
        print('max SWC in ml       : NaN')
    else:
        print('max SWC in ml 0th EM: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.transpose(avg_SWC_ml[0])[:25,:]), 
                                                                            np.asarray(h_avg[0])[ml1,ml2], 
                                                                            datetime.datetime.utcfromtimestamp(time_ml[0][ml1]).hour))
            
        
        

    val = np.zeros(shape=(10,np.asarray(avg_SWC_ml[0]).shape[0]))
#    val2= np.zeros(shape=(10,np.asarray(SWP_model[0]).shape[0]))
#    ice_EM = []
    ice_mean = []
    SWP_mean_ml = []
 #   ice_median = []
  #  ice_75 = []
   # ice_25 = []
    #ice_max = []
#    ice_min = []
    ice_std = []
    variable = []
# calculate the ensemble mean of ice amount (all 10 ensemble member averaged)
    for k in range(0,ice_amount_ml[0].shape[0]):
#    for k in range(0,25):
#    for k in range(0,1):
        for ens_memb in range(0,10):
            val[ens_memb] = (np.transpose(avg_SWC_ml[ens_memb])[k,:])
#            val2[ens_memb]= (np.transpose(SWP_model[ens_memb])[:])

        variable.append(val)
    ### std, mean
        ice_std.append(np.nanstd(val,axis=0,ddof=1))  
        ice_mean.append(np.nanmean(val,axis=0))
#        SWP_mean_ml = (np.nanmean(val2,axis=0))

### set ice mean values smaller zero to nan ###
    if len(ice_mean) == 0:
        continue
    else:
        a4,b4 = np.where(np.asarray(ice_mean) < 0.0)
        ice_mean = np.asarray(ice_mean)
        ice_mean[a4[:],b4[:]] = np.nan

### standard deviation over mean: ######
    std_mean = 100.*np.asarray(ice_std)/np.asarray(ice_mean)

    
    

#MEPS
    ml11, ml22 = np.where((ice_std) == np.nanmax(np.asarray(ice_std)[:Xmax+1,:] ))
    if len(ml1) == 0 or len(ml2) == 0:
        print('max std SWC in ml    : NaN')
    else:
        print('max std SWC in ml    : %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.asarray(ice_std)[:Xmax+1]), 
                                                                             np.asarray(h_avg[0])[ml11,ml22], 
                                                                             datetime.datetime.utcfromtimestamp(time_ml[0][ml11]).hour))

    ml101, ml202 = np.where((ice_mean) == np.nanmax(np.asarray(ice_mean)[:Xmax+1,:] ))
    if len(ml101) == 0 or len(ml202) == 0:
        print('max mean SWC in ml   : NaN')
    else:
        print('max mean SWC in ml   : %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.asarray(ice_mean)[:Xmax+1]), 
                                                                             np.asarray(h_avg[0])[ml101,ml202], 
                                                                             datetime.datetime.utcfromtimestamp(time_ml[0][ml101]).hour))
    ml3, ml4 = np.where((std_mean) == np.nanmax(np.asarray(std_mean)[:Xmax+1,:] ))
    if len(ml3) == 0 or len(ml4) == 0:
        print('max std/mean in ml   : NaN')
    else:
        print('max std/mean in ml   : %.2f  %.1f m; @ %.02d UTC' %(np.nanmax(np.asarray(std_mean)[:Xmax+1]), 
                                                                             np.asarray(h_avg[0])[ml3,ml4], 
                                                                             datetime.datetime.utcfromtimestamp(time_ml[0][ml3]).hour))



#    ml13, ml23 = np.where((ice_mean) == np.nanmax(np.asarray(ice_std)[:25,:] ))
 #   if len(ml1) == 0 or len(ml2) == 0:
  #      print('max std in ml EM     : NaN')
   # else:
    #    print('max std in ml EM mean: %.2f kg m^-3 in %.1f m; @ %.02d UTC' %(np.nanmax(np.asarray(ice_std)[:25,:]), 
     #                                                                        np.asarray(h_avg[0])[ml11,ml22], 
      #                                                                       datetime.datetime.utcfromtimestamp(time_ml[0][ml11]).hour))



    for ens_memb in range(0,10):
        if len(ice_amount_ml[ens_memb]) == 0:
            continue
        else:
            a2,b2 = np.where(ice_amount_ml[ens_memb][:,:] < 0.01)    ## 0.001? retrieved_snowfall_amount = iwcpsd*0.85*e-3*3600*24
            ice_amount_ml[ens_memb][a2[:],b2[:]] = np.nan
        if len(avg_SWC_ml[ens_memb]) == 0:
            continue
        else:
            a3,b3 = np.where(np.asarray(avg_SWC_ml[ens_memb]) < 0.01)
            avg_SWC_ml[ens_memb] = np.asarray(avg_SWC_ml[ens_memb])
            avg_SWC_ml[ens_memb][a3[:],b3[:]] = np.nan 
            
            a4,b4 = np.where(np.asarray(ice_mean) < 0.01)
            ice_mean = np.asarray(ice_mean)
            ice_mean[a4[:],b4[:]] = np.nan


        
        
# get only valuable values not nan    
        time_ml[ens_memb]       = (time_ml[ens_memb][~np.isnan(time_ml[ens_memb])])
        time[ens_memb]          = (td[:,~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1)])
        height[ens_memb]        = (height_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        thickness_ml[ens_memb]  = (thickness_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:]) 
        air_temp_ml[ens_memb]   = (air_temp_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        pressure_ml[ens_memb]   = (pressure_ml[ens_memb][~np.isnan(avg_SWC_ml[ens_memb]).any(axis=1),:])
        avg_SWC_ml[ens_memb]    = (avg_SWC_ml[ens_memb][:,~np.isnan(avg_SWC_ml[ens_memb]).any(axis=0)])
        
    time_std    = np.transpose(t_avg[0])[~np.isnan(np.asarray(ice_std)[:,:]).any(axis=1),0]
    height_std  = np.asarray(h_avg[0])[~np.isnan(np.asarray(ice_std)[:,:]).any(axis=1),:]      
    ice_std     = np.asarray(ice_std)[~np.isnan(np.asarray(ice_std)[:,:]).any(axis=1),:]



    
### plot ###########################################
    xdays = ['%s-%s-%s' %(year,month,ini_day), 
        6,12, 18,
        '%s-%s-%s' %(year,month,ini_day+1), 
        6, 12, 18,
        '%s-%s-%s' %(year,month,ini_day+2)]
    
    
    fig_name = '%s%s%s.%s' %(year,month,ini_day,form)

#    fig = plt.figure(1, figsize=(20,7))
    fig = plt.figure(1, figsize=(30,10))
    ax = fig.add_subplot(111)
    # Vertical line to show end of day
    ax.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax.axvline(24,color = spagh.vert_col, linewidth = 3)
    ax.axvline(48,color = spagh.vert_col, linewidth = 3)

#    levels = np.arange(0,1.51,0.01)
#    levels = np.arange(0.1,1.41,0.1)
    levels = np.arange(0.1,1.51,0.1)
    v_min=0.
#    v_max=1.4
    v_max=1.5
    ymax=3000.
#    xticks=im.xticks2
    xticks=np.arange(0,49,6)
    yticks=im.yticks1
#    xlabel=im.times
    xlabel=xdays
    

#    im0 = ax.contourf(np.transpose(t_avg[0])[:Xmax+1,:], np.asarray(h_avg[0])[:Xmax+1,:], np.asarray(ice_std)[:Xmax+1,:],
 #                     levels,
  #          cmap='magma', #extend = 'max', 
   #                   alpha = 1.0, 
    #                   vmin = v_min, vmax = v_max, origin = 'lower')
#    im0 = ax.contourf(time_std, height_std[0,:],np.transpose(ice_std),levels,
 #                     cmap='magma', #extend = 'max', 
  #                    alpha = 1.0, 
   #                    vmin = v_min, vmax = v_max, origin = 'lower')
#    levels3 = np.arange(0,110,10)
 #   levels3 = [0, 30, 50, 70, 90, 100]
    levels3 = [0, 25, 50, 75, 100]
    im0 = ax.contourf(np.transpose(t_avg[0])[:Xmax+1,:], np.asarray(h_avg[0])[:Xmax+1,:],
                      std_mean[:Xmax+1,:],levels3,
                      #cmap='seismic', extend = 'max', 
                      cmap='magma', extend = 'max', 
                      alpha = 1.0, 
                       vmin = v_min, vmax = 100., origin = 'lower')
    
    
#    thlevs = np.arange(0,1.51,0.1)
    levels2 = np.arange(0,1.41,0.1)
    cth = ax.contour(np.transpose(t_avg[0])[:Xmax+1,:], np.asarray(h_avg[0])[:Xmax+1,:], np.asarray(ice_mean)[:Xmax+1,:], 
                     levels2, colors= 'grey', 
                linewidths=2.)
    plt.clabel(cth, fontsize = 18.+2, inline=1, fmt = '%0.1f')

# set the limits of the plot to the limits of the data
    ax.axis([0., Xmax, 0., ymax])

# labels 
    im.labels_x(ax,xticks,xlabel)
    im.labels_y(ax,yticks,im.yl1,'height [km]')
# add colorbar   
#    im.add_colorbar(fig,im0, ax, levels3[::2] ,var_label='coefficient of variation [%]')
    im.add_colorbar(fig,im0, ax, levels3 ,var_label='coefficient of variation [%]')


    labels = ['ensemble mean']
    for i in range(len(labels)):
        cth.collections[i].set_label(labels[i])

    lgd = ax.legend(loc='upper left',fontsize=im.label_fs+2)
    frame = lgd.get_frame()
    frame.set_facecolor('white')

    if hh < 10:
        hh = '0%s' %(hh)
    else:
        hh = '%s' %(hh)
# title
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    ax.set_title(title, fontsize=im.fontsize+2, color =im.date_blue )
# tight layout
    fig.tight_layout(pad=1.4,  h_pad=2.5)
    fig.subplots_adjust(top=0.94)    
    
    
    if savefig ==1:
        sF.save_figure_portrait(figdir, fig_name, form)
        print('saved: %s/%s' %(figdir, fig_name))
    else:
        plt.show()
    plt.close()
    
    for ens_memb in range(0,10):
        fn_snow[ens_memb].close()
        fn_air_temp[ens_memb].close() 
        fn_cl_ice[ens_memb].close

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in greater_equal
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: Mean of empty slice
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:100: RuntimeWarning: invalid value encountered in greater


Tuesday, 20 Dec 2016
max SWC in retrieval: NaN
max SWC in ml 0th EM: 0.04 kg m^-3 in 400.0 m; @ 10 UTC
max std SWC in ml    : 0.50 kg m^-3 in 1600.0 m; @ 23 UTC
max mean SWC in ml   : 0.36 kg m^-3 in 1600.0 m; @ 23 UTC
max std/mean in ml   : 18169.40  1600.0 m; @ 22 UTC


/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:110: RuntimeWarning: All-NaN slice encountered
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:152: RuntimeWarning: Mean of empty slice
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:159: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:209: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:214: RuntimeWarning: invalid value encountered in less
/Volumes/SANDISK128/App

saved: ../../Figures/Retrieval_MEPS/coefficent_variation/_48/20161220.png
Wednesday, 21 Dec 2016
max SWC in retrieval: 1.08 kg m^-3 in 600.0 m; @ 16 UTC
max SWC in ml 0th EM: 2.11 kg m^-3 in 1400.0 m; @ 20 UTC
max std SWC in ml    : 1.64 kg m^-3 in 1800.0 m; @ 14 UTC
max mean SWC in ml   : 1.24 kg m^-3 in 1400.0 m; @ 20 UTC
max std/mean in ml   : 4335.65  1800.0 m; @ 01 UTC
saved: ../../Figures/Retrieval_MEPS/coefficent_variation/_48/20161221.png
Thursday, 22 Dec 2016
max SWC in retrieval: 1.46 kg m^-3 in 1200.0 m; @ 10 UTC
max SWC in ml 0th EM: 1.35 kg m^-3 in 1400.0 m; @ 12 UTC
max std SWC in ml    : 1.36 kg m^-3 in 1400.0 m; @ 14 UTC
max mean SWC in ml   : 0.99 kg m^-3 in 1400.0 m; @ 14 UTC
max std/mean in ml   : 192.13  3000.0 m; @ 03 UTC
saved: ../../Figures/Retrieval_MEPS/coefficent_variation/_48/20161222.png
Friday, 23 Dec 2016
max SWC in retrieval: 0.91 kg m^-3 in 600.0 m; @ 23 UTC
max SWC in ml 0th EM: 0.54 kg m^-3 in 400.0 m; @ 20 UTC
max std SWC in ml    : 0.46 kg m^-3 in 18